In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tabular-playground-series-jun-2021/sample_submission.csv
/kaggle/input/tabular-playground-series-jun-2021/train.csv
/kaggle/input/tabular-playground-series-jun-2021/test.csv


In [3]:
#2
train = pd.read_csv("/kaggle/input/tabular-playground-series-jun-2021/train.csv")
test = pd.read_csv("/kaggle/input/tabular-playground-series-jun-2021/test.csv")
ss = pd.read_csv("/kaggle/input/tabular-playground-series-jun-2021/sample_submission.csv")

In [4]:
#3
from sklearn.datasets import make_classification
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import StratifiedShuffleSplit
import pandas as pd
from pandas_profiling import ProfileReport

def imbalanced_data_split(X, y, test_size=0.2):
    sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)
    for train_index, test_index in sss.split(X, y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        return X_train, X_test, y_train, y_test


X = train.drop(["id","target"], axis=1)
y = train.target
unchi = test.drop(["id"], axis=1)

X_train, X_test, y_train, y_test = imbalanced_data_split(X, y, test_size=0.2)
# for validation
X_train2, X_valid, y_train2, y_valid = imbalanced_data_split(X_train, y_train, test_size=0.2)

In [5]:
# 普通のLGBM
import lightgbm as lgb
from sklearn.metrics import roc_auc_score

lgbm_params = {'objective': 'multiclass',
 'metric': 'multi_logloss',
 #'boosting_type':"dart",
 'num_class': 9,
 'force_row_wise': True,
 'feature_pre_filter': False,
 'lambda_l1': 9.743033661523311,
 'lambda_l2': 2.709400532224499e-07,
 'num_leaves': 33,
 'feature_fraction': 0.4,
 'bagging_fraction': 1.0,
 'bagging_freq': 0,
 'min_child_samples': 100,
 'num_iterations': 100,
 'early_stopping_round': 5}


def lgbm_train(X_train_df, X_valid_df, y_train_df, y_valid_df, lgbm_params):
    lgb_train = lgb.Dataset(X_train_df, y_train_df)
    lgb_eval = lgb.Dataset(X_valid_df, y_valid_df, reference=lgb_train)

    # 上記のパラメータでモデルを学習する
    model = lgb.fit(lgbm_params, lgb_train,
                      valid_sets=lgb_eval,
                      num_boost_round=1000,
                      early_stopping_rounds=10)
    
    return model

In [6]:
# 目的変数Encode
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y_train2)
y_train2 = le.transform(y_train2)
y_train2 = pd.Series(y_train2)

le.fit(y_train)
y_train = le.transform(y_train)
y_train = pd.Series(y_train)

le.fit(y_test)
y_test = le.transform(y_test)
y_test = pd.Series(y_test)

le.fit(y_valid)
y_valid = le.transform(y_valid)
y_valid = pd.Series(y_valid)

In [7]:
# 普通のLGBM
model_normal = lgbm_train(X_train2, X_valid, y_train2, y_valid, lgbm_params)

/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Info] Total Bins 3274
[LightGBM] [Info] Number of data points in the train set: 128000, number of used features: 75
[LightGBM] [Info] Start training from score -3.088156
[LightGBM] [Info] Start training from score -2.102454
[LightGBM] [Info] Start training from score -2.603901
[LightGBM] [Info] Start training from score -3.749758
[LightGBM] [Info] Start training from score -4.178576
[LightGBM] [Info] Start training from score -1.350716
[LightGBM] [Info] Start training from score -2.605804
[LightGBM] [Info] Start training from score -1.351621
[LightGBM] [Info] Start training from score -2.057986
[1]	valid_0's multi_logloss: 1.87607
Training until validation scores don't improve for 5 rounds
[2]	valid_0's multi_logloss: 1.85862
[3]	valid_0's multi_logloss: 1.8438
[4]	valid_0's multi_logloss: 1.83125
[5]	valid_0's multi_logloss: 1.82079
[6]	valid_0's multi_logloss: 1.81157
[7]	valid_0's multi_logloss: 1.80408
[8]	valid_0's multi_logloss: 1.79761
[9]	valid_0's multi_logloss: 1.

In [9]:
lgbm_pred = model_normal.predict(unchi, num_iteration=model_normal.best_iteration)

In [10]:
ss["Class_1"] = lgbm_pred[:,0]
ss["Class_2"] = lgbm_pred[:,1]
ss["Class_3"] = lgbm_pred[:,2]
ss["Class_4"] = lgbm_pred[:,3]
ss["Class_5"] = lgbm_pred[:,4]
ss["Class_6"] = lgbm_pred[:,5]
ss["Class_7"] = lgbm_pred[:,6]
ss["Class_8"] = lgbm_pred[:,7]
ss["Class_9"] = lgbm_pred[:,8]
ss.to_csv("/kaggle/working/sub.csv", index=False)

downsampling

In [11]:
from imblearn.under_sampling import RandomUnderSampler

sampler = RandomUnderSampler(random_state=42)
# downsampling
X_resampled, y_resampled = sampler.fit_resample(X_train, y_train)
# for validation
X_train2, X_valid, y_train2, y_valid = imbalanced_data_split(X_resampled, y_resampled, test_size=0.2)

In [12]:
model_ds = lgbm_train(X_train2, X_valid, y_train2, y_valid, lgbm_params)

/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Info] Total Bins 3126
[LightGBM] [Info] Number of data points in the train set: 17647, number of used features: 75
[LightGBM] [Info] Start training from score -2.197621
[LightGBM] [Info] Start training from score -2.197111
[LightGBM] [Info] Start training from score -2.197111
[LightGBM] [Info] Start training from score -2.197111
[LightGBM] [Info] Start training from score -2.197111
[LightGBM] [Info] Start training from score -2.197621
[LightGBM] [Info] Start training from score -2.197111
[LightGBM] [Info] Start training from score -2.197111
[LightGBM] [Info] Start training from score -2.197111
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1]	valid_0's multi_logloss: 2.18568
Training until validation scores don't improve for 5 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best g

2.10 ゴミ

bagging

In [13]:
def bagging(seed):
    sampler = RandomUnderSampler(random_state=seed, replacement=True)
    X_resampled, y_resampled = sampler.fit_resample(X_train, y_train)
    X_train2, X_valid, y_train2, y_valid = imbalanced_data_split(X_resampled, y_resampled, test_size=0.2)
    model_bagging = lgbm_train(X_train2, X_valid, y_train2, y_valid, lgbm_params)
    return model_bagging

In [14]:
models = []

for i in range(10):
    models.append(bagging(i))

/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Info] Total Bins 3124
[LightGBM] [Info] Number of data points in the train set: 17647, number of used features: 75
[LightGBM] [Info] Start training from score -2.197621
[LightGBM] [Info] Start training from score -2.197111
[LightGBM] [Info] Start training from score -2.197111
[LightGBM] [Info] Start training from score -2.197111
[LightGBM] [Info] Start training from score -2.197111
[LightGBM] [Info] Start training from score -2.197621
[LightGBM] [Info] Start training from score -2.197111
[LightGBM] [Info] Start training from score -2.197111
[LightGBM] [Info] Start training from score -2.197111
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1]	valid_0's multi_logloss: 2.18355
Training until validation scores don't improve for 5 rounds
[2]	valid_0's multi_logloss: 2.1726
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3]	valid_0's multi_logloss: 2.1618
[LightGBM] [Warning] No further splits with positive gain, best gain

/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Info] Total Bins 3118
[LightGBM] [Info] Number of data points in the train set: 17647, number of used features: 75
[LightGBM] [Info] Start training from score -2.197621
[LightGBM] [Info] Start training from score -2.197111
[LightGBM] [Info] Start training from score -2.197111
[LightGBM] [Info] Start training from score -2.197111
[LightGBM] [Info] Start training from score -2.197111
[LightGBM] [Info] Start training from score -2.197621
[LightGBM] [Info] Start training from score -2.197111
[LightGBM] [Info] Start training from score -2.197111
[LightGBM] [Info] Start training from score -2.197111
[1]	valid_0's multi_logloss: 2.18286
Training until validation scores don't improve for 5 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2]	valid_0's multi_logloss: 2.17072
[3]	valid_0's multi_logloss: 2.1602
[4]	valid_0's multi_logloss: 2.15069
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No furthe

/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


...

[LightGBM] [Info] Total Bins 3117
[LightGBM] [Info] Number of data points in the train set: 17647, number of used features: 75
[LightGBM] [Info] Start training from score -2.197621
[LightGBM] [Info] Start training from score -2.197111
[LightGBM] [Info] Start training from score -2.197111
[LightGBM] [Info] Start training from score -2.197111
[LightGBM] [Info] Start training from score -2.197111
[LightGBM] [Info] Start training from score -2.197621
[LightGBM] [Info] Start training from score -2.197111
[LightGBM] [Info] Start training from score -2.197111
[LightGBM] [Info] Start training from score -2.197111
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1]	valid_0's multi_logloss: 2.18352
Training until validation scores don't improve for 5 rounds
[2]	valid_0's multi_logloss: 2.17111
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3]	valid_0's multi_logloss: 2.16081
[LightGBM] [Warning] No further splits with positive gain, best ga

/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Info] Total Bins 3107
[LightGBM] [Info] Number of data points in the train set: 17647, number of used features: 75
[LightGBM] [Info] Start training from score -2.197621
[LightGBM] [Info] Start training from score -2.197111
[LightGBM] [Info] Start training from score -2.197111
[LightGBM] [Info] Start training from score -2.197111
[LightGBM] [Info] Start training from score -2.197111
[LightGBM] [Info] Start training from score -2.197621
[LightGBM] [Info] Start training from score -2.197111
[LightGBM] [Info] Start training from score -2.197111
[LightGBM] [Info] Start training from score -2.197111
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1]	valid_0's multi_logloss: 2.18436
Training until validation scores don't improve for 5 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2]	valid_0's multi_logloss: 2.17332
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3]	valid_0's multi_logloss:

/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Info] Total Bins 3096
[LightGBM] [Info] Number of data points in the train set: 17647, number of used features: 75
[LightGBM] [Info] Start training from score -2.197621
[LightGBM] [Info] Start training from score -2.197111
[LightGBM] [Info] Start training from score -2.197111
[LightGBM] [Info] Start training from score -2.197111
[LightGBM] [Info] Start training from score -2.197111
[LightGBM] [Info] Start training from score -2.197621
[LightGBM] [Info] Start training from score -2.197111
[LightGBM] [Info] Start training from score -2.197111
[LightGBM] [Info] Start training from score -2.197111
[1]	valid_0's multi_logloss: 2.18317
Training until validation scores don't improve for 5 rounds
[2]	valid_0's multi_logloss: 2.17114
[3]	valid_0's multi_logloss: 2.15997
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4]	valid_0's multi_logloss: 2.15181
[LightGBM] [Warning] No furth